# Discrete Wavelet Transform (DWT)

Discrete Wavelet Transform (DWT) was applied to extract time-frequency features across delta, theta, alpha, and beta bands. Both FFT and DWT features were stored for each subject.

In [3]:
import mne
import numpy as np
import pandas as pd
import os
import pywt

BASE_DIR = r"C:\Users\User\Documents\EEG_Project\rEEG"
SUBJ = "sub-001"
SUBJ_DIR = os.path.join(BASE_DIR, SUBJ)
DATA_DIR = os.path.join(SUBJ_DIR, "data")
EPO_FILE = os.path.join(SUBJ_DIR, "epo_001_raw.fif")

epochs = mne.read_epochs(EPO_FILE, preload=True)
sfreq = epochs.info["sfreq"]
ch_names = epochs.ch_names
data = epochs.get_data()

WAVELET = 'db4'
LEVEL = 6 

# Mapping of Detail Coefficients (D) to Physiological Bands (approx for fs=250)
# --- DWT per epoch/channel ---
bands_dwt = {
    "delta": 6,   # A6 approx 0.5–4 Hz
    "theta": 5,   # D5 approx 4–8 Hz
    "alpha": 4,   # D4 approx 8–16 Hz
    "beta": 3     # D3 approx 16–32 Hz
}
# Container for results
rows_dwt = []

for e_idx, epoch in enumerate(data):
    for c_idx, ch_data in enumerate(epoch):
        # Compute wavelet decomposition
        coeffs = pywt.wavedec(ch_data, wavelet=WAVELET, level=LEVEL)
        # coeffs[0] is the approximation at level 6 (lowest freq), coeffs[1] = D6, etc.
        entry = {'epoch': e_idx, 'channel': ch_names[c_idx]}
        for band, level_idx in bands_dwt.items():
            if band == "delta":
                # Approximation coefficients A6
                coeff = coeffs[0]
            else:
                # Detail coefficients D3–D5
                coeff = coeffs[LEVEL - level_idx + 1]
            # Feature: energy of coefficients
            energy = np.sum(np.square(coeff))
            entry[f"{band}_energy"] = energy
        rows_dwt.append(entry)

# Save to CSV
os.makedirs(DATA_DIR, exist_ok=True)
df_dwt = pd.DataFrame(rows_dwt)
df_dwt.to_csv(os.path.join(DATA_DIR, f"{SUBJ}_DWT_band_energy.csv"), index=False)

print(f"DWT features saved: ‼️{os.path.join(DATA_DIR, f'{SUBJ}_DWT_band_energy.csv')}")




Reading C:\Users\User\Documents\EEG_Project\rEEG\sub-001\epo_001_raw.fif ...
    Found the data of interest:
        t =       0.00 ...    4000.00 ms
        0 CTF compensation matrices available
Not setting metadata
77 matching events found
No baseline correction applied
0 projection items activated


C:\Users\User\AppData\Local\Temp\ipykernel_28800\1391504496.py:13: RuntimeWarning: This filename (C:\Users\User\Documents\EEG_Project\rEEG\sub-001\epo_001_raw.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(EPO_FILE, preload=True)


DWT features saved: ‼️C:\Users\User\Documents\EEG_Project\rEEG\sub-001\data\sub-001_DWT_band_energy.csv


In [4]:
import mne
import numpy as np
import pandas as pd
import os
import pywt

BASE_DIR = r"C:\Users\User\Documents\EEG_Project\rEEG"

WAVELET = 'db4'
LEVEL = 6  # decomposition level

# Explicit mapping of EEG bands to DWT coefficients
# A6 = coeffs[0], D6 = coeffs[1], D5 = coeffs[2], D4 = coeffs[3], D3 = coeffs[4]
coeff_indices = {
    "delta": [0, 1],  # A6 + D6
    "theta": [2],     # D5
    "alpha": [3],     # D4
    "beta": [4]       # D3
}

for subj_num in range(1, 150):  # subjects 001–045
    SUBJ = f"sub-{subj_num:03d}"
    SUBJ_DIR = os.path.join(BASE_DIR, SUBJ)
    DATA_DIR = os.path.join(SUBJ_DIR, "data")
    EPO_FILE = os.path.join(SUBJ_DIR, f"epo_{subj_num:03d}_raw.fif")

    if not os.path.exists(EPO_FILE):
        print(f"File not found: {EPO_FILE}, skipping.")
        continue

    print(f"Processing {SUBJ}...")

    epochs = mne.read_epochs(EPO_FILE, preload=True)
    ch_names = epochs.ch_names
    data = epochs.get_data()  # shape: (n_epochs, n_channels, n_times)

    rows_dwt = []

    for e_idx, epoch in enumerate(data):
        for c_idx, ch_data in enumerate(epoch):
            coeffs = pywt.wavedec(ch_data, wavelet=WAVELET, level=LEVEL)
            entry = {'epoch': e_idx, 'channel': ch_names[c_idx]}
            for band, inds in coeff_indices.items():
                # Sum energy across all coefficients assigned to the band
                energy = sum(np.sum(np.square(coeffs[i])) for i in inds)
                entry[f"{band}_energy"] = energy
            rows_dwt.append(entry)

    os.makedirs(DATA_DIR, exist_ok=True)
    df_dwt = pd.DataFrame(rows_dwt)
    out_file = os.path.join(DATA_DIR, f"{SUBJ}_DWT_band_energy.csv")
    df_dwt.to_csv(out_file, index=False)
    print(f"DWT features saved: {out_file}")


Processing sub-001...
Reading C:\Users\User\Documents\EEG_Project\rEEG\sub-001\epo_001_raw.fif ...
    Found the data of interest:
        t =       0.00 ...    4000.00 ms
        0 CTF compensation matrices available
Not setting metadata
77 matching events found
No baseline correction applied
0 projection items activated


C:\Users\User\AppData\Local\Temp\ipykernel_28800\3745407761.py:33: RuntimeWarning: This filename (C:\Users\User\Documents\EEG_Project\rEEG\sub-001\epo_001_raw.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(EPO_FILE, preload=True)


DWT features saved: C:\Users\User\Documents\EEG_Project\rEEG\sub-001\data\sub-001_DWT_band_energy.csv
Processing sub-002...
Reading C:\Users\User\Documents\EEG_Project\rEEG\sub-002\epo_002_raw.fif ...
    Found the data of interest:
        t =       0.00 ...    4000.00 ms
        0 CTF compensation matrices available
Not setting metadata
162 matching events found
No baseline correction applied
0 projection items activated


C:\Users\User\AppData\Local\Temp\ipykernel_28800\3745407761.py:33: RuntimeWarning: This filename (C:\Users\User\Documents\EEG_Project\rEEG\sub-002\epo_002_raw.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(EPO_FILE, preload=True)


DWT features saved: C:\Users\User\Documents\EEG_Project\rEEG\sub-002\data\sub-002_DWT_band_energy.csv
Processing sub-003...
Reading C:\Users\User\Documents\EEG_Project\rEEG\sub-003\epo_003_raw.fif ...
    Found the data of interest:
        t =       0.00 ...    4000.00 ms
        0 CTF compensation matrices available
Not setting metadata
125 matching events found
No baseline correction applied
0 projection items activated


...

DWT features saved: C:\Users\User\Documents\EEG_Project\rEEG\sub-147\data\sub-147_DWT_band_energy.csv
Processing sub-148...
Reading C:\Users\User\Documents\EEG_Project\rEEG\sub-148\epo_148_raw.fif ...
    Found the data of interest:
        t =       0.00 ...    4000.00 ms
        0 CTF compensation matrices available
Not setting metadata
78 matching events found
No baseline correction applied
0 projection items activated


C:\Users\User\AppData\Local\Temp\ipykernel_28800\3745407761.py:33: RuntimeWarning: This filename (C:\Users\User\Documents\EEG_Project\rEEG\sub-148\epo_148_raw.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(EPO_FILE, preload=True)


DWT features saved: C:\Users\User\Documents\EEG_Project\rEEG\sub-148\data\sub-148_DWT_band_energy.csv
Processing sub-149...
Reading C:\Users\User\Documents\EEG_Project\rEEG\sub-149\epo_149_raw.fif ...
    Found the data of interest:
        t =       0.00 ...    4000.00 ms
        0 CTF compensation matrices available
Not setting metadata
63 matching events found
No baseline correction applied
0 projection items activated


C:\Users\User\AppData\Local\Temp\ipykernel_28800\3745407761.py:33: RuntimeWarning: This filename (C:\Users\User\Documents\EEG_Project\rEEG\sub-149\epo_149_raw.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(EPO_FILE, preload=True)


DWT features saved: C:\Users\User\Documents\EEG_Project\rEEG\sub-149\data\sub-149_DWT_band_energy.csv
